In [1]:

!pip install flask_ngrok
!pip install pyngrok==4.1.1
!pip install Werkzeug

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=89a9b18b7a24f1570fdb9d24125a11c2fbf58204a7dffd5abaffcd5694f2cb07
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [2]:
import time

def run(instance_prompt,class_prompt,training_steps):
  HUGGINGFACE_TOKEN = ""

#markdown Name/Path of the initial model.
  MODEL_NAME = "runwayml/stable-diffusion-v1-5" #param {type:"string"}
  BRANCH = "fp16" #param {type:"string"}

#markdown Enter instance prompt and class prompt.\
#markdown Example 1: photo of zwx person, photo of a person\
#markdown Example 2: photo of zwx toy, photo of a toy
  instance_prompt = instance_prompt #param {type:"string"}
  class_prompt =  class_prompt #param {type:"string"}
  training_steps = training_steps #param {type:"integer"}
  learning_rate = 1e-6
  A=class_prompt.split()
  output_file = "AI_PICS/models/"+str(A[0])+"_model.safetensors" #param {type:"string"}
#markdown  Convert to fp16? (takes half the space (2GB)).
  fp16 = True #param {type: "boolean"}

#markdown Clear log after run?
  CLEAR_LOG = False #param {type:"boolean"}

# Mount google drive for saving the model
  from google.colab import drive
  drive.mount('/content/drive')

  OUTPUT_DIR = "/content/output"
  INSTANCE_DIR = "/content/data/instance"
  CLASS_DIR = "/content/data/class"

  !mkdir -p $INSTANCE_DIR
  !mkdir -p $CLASS_DIR

  !rm -rf $OUTPUT_DIR
  !mkdir -p $OUTPUT_DIR

  if 'pipe' in locals():
    del pipe

# Check type of GPU and VRAM available.
#!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader
  !nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

# Upload training images.
  import os
  from google.colab import files
  import shutil

  time_start = time.time()
  def clear():
      from IPython.display import clear_output; return clear_output()

# huggingface token
  !mkdir -p ~/.huggingface
  !echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

# install diffusers
  !git clone https://github.com/sagiodev/diffusers.git #https://github.com/huggingface/diffusers.git
  %cd diffusers
  !git checkout 08b453e3828f80027d881bb460716af95e192bcd -- ./scripts/convert_diffusers_to_original_stable_diffusion.py
  %pip install .


# install dreambooth
  %cd /content/diffusers/examples/dreambooth
  %pip install -r requirements.txt
  %pip install bitsandbytes #xformers torch==2.1.0+cu121 # gradio natsort safetensors xformers torch==2.1.0+cu121 accelerate
  !accelerate config default


# test bitsandbytes
  import bitsandbytes
  import torch
  print(torch.__version__)

# Dreambooth training - Edit this section to customize parameters
  !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --revision=$BRANCH \
    --instance_prompt="$instance_prompt" \
    --class_prompt="$class_prompt" \
    --class_data_dir=$CLASS_DIR \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --with_prior_preservation --prior_loss_weight=1.0 \
    --seed=1337 \
    --resolution=512 \
    --train_batch_size=1 \
    --train_text_encoder \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=$learning_rate \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --num_class_images=50 \
    --sample_batch_size=4 \
    --max_train_steps=$training_steps

#  --enable_xformers_memory_efficient_attention \

########################################
#prompt = "photo of zwx"
  prompt = instance_prompt
  negative_prompt = ""
  num_samples = 4
  guidance_scale = 7.5
  num_inference_steps = 25
  height = 512
  width = 512
  seed = 100

  %cd /content/
  from diffusers import StableDiffusionPipeline,EulerDiscreteScheduler
  from matplotlib.pyplot import figure, imshow, axis
  from matplotlib.image import imread
  import numpy as np



  if 'pipe' not in locals():
    pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    g_cuda = None




  g_cuda = torch.Generator(device='cuda')

  g_cuda.manual_seed(seed)

  from torch import autocast
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images

      fig=figure(figsize=(5*num_samples,5*num_samples))
      for i in range(num_samples):
          a=fig.add_subplot(1, num_samples,i+1)
          imshow(images[i])
          axis('off')


  ######################################
  # Save to checkpoint file
  ckpt_path = '/content/drive/MyDrive/Data_file'+ output_file
  # save model file. Save to a new name if already exist.
  dirname= os.path.dirname(ckpt_path)
  !mkdir -p $dirname
  filename = os.path.basename(ckpt_path)
  fileanmeWithoutExt = os.path.splitext(filename)[0]
  ExtName = os.path.splitext(filename)[1]
  filenamePattern = fileanmeWithoutExt + '%d' + ExtName
  i = 1
  while os.path.isfile(ckpt_path):
    filename = filenamePattern%i
    ckpt_path = dirname + '/' + filename
    i += 1

  half_arg = "--half" if fp16 else ""
  !python /content/diffusers/scripts/convert_diffusers_to_original_stable_diffusion.py \
      --use_safetensors \
      --model_path $OUTPUT_DIR \
      --checkpoint_path $ckpt_path $half_arg
  print(f"[*] Converted ckpt saved at {ckpt_path}")
  minutes = (time.time()-time_start)/60
  print("Dreambooth completed successfully. It took %1.1f minutes."%minutes)

In [3]:
#@title Test image generation from model
def run2(prompt,negative_prompt,num_samples ,
  guidance_scale ,
  height ,
  width ):

  import bitsandbytes
  import torch
  seed = -1
  num_inference_steps = 50

  %cd /content/
  from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
  from matplotlib.pyplot import figure, imshow, axis
  from matplotlib.image import imread
  import numpy as np
  OUTPUT_DIR = "/content/output"
  INSTANCE_DIR = "/content/data/instance"
  CLASS_DIR = "/content/data/class"
  if 'pipe' not in locals():
    pipe = StableDiffusionPipeline.from_pretrained(OUTPUT_DIR, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    g_cuda = None



  g_cuda = torch.Generator(device='cuda')

  g_cuda.manual_seed(seed)

  from torch import autocast
  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          prompt,
          height=height,
          width=width,
          negative_prompt=negative_prompt,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images
      output_directory = "/content/drive/MyDrive/Colab Notebooks/FYP/static"
      i=1
      for im in images:
        filename = "generated_image_"+str(i)+".png"
        print(filename)
        filepath = os.path.join(output_directory, filename)
        # Save the image
        im.save(filepath)
        i=i+1
        print()

    # Optionally, display the saved filepath
      print(f"Image saved to: {filepath}")
      from ipywidgets import widgets, HBox
      from IPython.display import display
      for im in images:
          display(im)


In [4]:
!ngrok authtoken 2bOwBvJflwp2aPa7zT2IoqNHnCa_2YEHRHEzY6uraDR4yhpgP
from google.colab import files
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok as r
import shutil
from werkzeug.utils import secure_filename
import os
from google.colab import drive
drive.mount('/content/drive')

# Specify the path of the folder you want to create
folder_path = '/content/data/instance'

# Check if the folder already exists
if not os.path.exists(folder_path):
    # If not, create the folder
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created successfully.")
else:
    print(f"Folder '{folder_path}' already exists.")

templates_folder = 'drive/MyDrive/Colab Notebooks/FYP/templates'
static_folder = 'drive/MyDrive/Colab Notebooks/FYP/static'
# image_folder = 'drive/MyDrive/Colab Notebooks/FYP/image'
app = Flask(__name__,template_folder = templates_folder,static_folder = static_folder)

@app.route('/')
def index():
  return render_template('index.html')

@app.route('/add', methods=['POST'])
def add_numbers():
    try:
          !ngrok authtoken 2bOwBvJflwp2aPa7zT2IoqNHnCa_2YEHRHEzY6uraDR4yhpgP
          folder_path='/content/data/instance'
          files = os.listdir(folder_path)

    # Iterate over the files and delete each one
          for file in files:
            file_path = os.path.join(folder_path, file)
            try:
              os.remove(file_path)
              print(f"File '{file_path}' deleted successfully.")
            except Exception as e:
              print(f"Error deleting file '{file_path}': {e}")
          INSTANCE_DIR = "/content/data/instance"
          iprompt = str(request.form['iprompt'])
          cprompt = str(request.form['cprompt'])
          steps = int(request.form['steps'])
          file0= request.files.getlist('files')
          print(file0)
          for x in file0:
            filename=secure_filename(x.filename)
            x.save(os.path.join(INSTANCE_DIR,filename))
            #run(iprompt,cprompt,steps)
          return render_template('result.html')

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return render_template('error.html', message=error_message)


@app.route('/add0', methods=['POST'])
def two_page():
  try :
    !ngrok authtoken 2bOwBvJflwp2aPa7zT2IoqNHnCa_2YEHRHEzY6uraDR4yhpgP
    prompt = str(request.form['prompt'])
    nprompt = str(request.form['nprompt'])
    sample = int(request.form['sample'])
    gs = float(request.form['gs'])
    height = int(request.form['height'])
    width = int(request.form['width'])
    run2(prompt,nprompt,sample,gs,height,width)
     #image_filenames=[]
     #for x in range(1,sample+1):
      # image_filenames = image_filenames.append(f"ajs_{i+1}-2.png")
    image_filenames=[]
    for x in range(1,sample+1):
      image_filenames.append("generated_image_"+str(x)+".png")
    # Construct the full path for each image
    return render_template('final.html', image_filenames=image_filenames)
  except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return render_template('error.html', message=error_message)
r(app)
app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder '/content/data/instance' created successfully.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://29ba-34-90-24-47.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
